In [ ]:
from mdgen.parsing import parse_train_args

from mdgen.logger import get_logger
logger = get_logger(__name__)

import torch, os
from mdgen.dataset import LatentDataset



torch.set_float32_matmul_precision('medium')
# args = parse_train_args()
    

trainset = LatentDataset("test/rcut3.5_energy_encodedim4_perturbeddata/encoded_dataset-clean/", 3.5, num_frames=1, stage="train", random_starting_point=True)

In [ ]:

for i in range(len(trainset)):
    print(i, trainset[i]['v'].shape)


In [ ]:
batch_size = 1
train_loader = torch.utils.data.DataLoader(
    trainset,
    batch_size=batch_size,
    num_workers=0,
    shuffle=True,
)
sample_batch = next(iter(train_loader))

## Check latent flow

In [ ]:


from argparse import Namespace
args_latentFlow = Namespace(
    sim_ckpt="workdir/vlatent-NoiseAwareModel-path_linearOT/epoch=2659-step=891100.ckpt",
    data_dir="test/rcut3.5_energy_encodedim4_perturbeddata/encoded_dataset-clean/",
    suffix="",
    num_frames=1,
    num_rollouts=1,
    # out_dir="./test/localmask_rcut3.5_loss_regress-path_linearOT",
    out_dir="./test/rcut3.5_decoder_embeddim4_perturbeddata",
    random_starting_point=True,
    localmask=False,
    )

In [ ]:
os.makedirs(args_latentFlow.out_dir, exist_ok=True)

In [ ]:
from mdgen.latentflow_wrapper import LatentGenWrapper


In [ ]:
device='cpu'
ckpt = torch.load(args_latentFlow.sim_ckpt, weights_only=False)
model_latentFlow = LatentGenWrapper(**ckpt["hyper_parameters"])
model_latentFlow.load_state_dict(ckpt["state_dict"], strict=False)
model_latentFlow.eval().to(device)

In [ ]:
print(sample_batch.keys())

In [ ]:
for key in ['v', 'h']:
    sample_batch[key] = sample_batch[key].to(device)
# prep_latentFlow = model_latentFlow.prep_batch(sample_batch)

pred_latent, _ = model_latentFlow.inference(sample_batch)

In [ ]:
print(pred_latent.shape)

In [ ]:
print(sample_batch['h'].shape)

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(2,2,figsize=(7, 7))
axes[0][0].scatter(sample_batch['v'][0,0,:,0,0].detach().numpy(), pred_latent[0,0,:,0,1].detach().numpy(), s=1)
axes[0][0].scatter(sample_batch['v'][0,0,:,1,0].detach().numpy(), pred_latent[0,0,:,1,1].detach().numpy(), s=1)
axes[0][0].scatter(sample_batch['v'][0,0,:,2,0].detach().numpy(), pred_latent[0,0,:,2,1].detach().numpy(), s=1)
axes[0][0].scatter(sample_batch['v'][0,0,:,3,0].detach().numpy(), pred_latent[0,0,:,3,1].detach().numpy(), s=1)
axes[0][0].set_xlabel("v")
axes[0][0].set_ylabel("$v^{pred}$")
axes[0][0].set_title("$v_x$")
axes[0][1].scatter(sample_batch['v'][0,0,:,0,1].detach().numpy(), pred_latent[0,0,:,0,2].detach().numpy(), s=1)
axes[0][1].scatter(sample_batch['v'][0,0,:,1,1].detach().numpy(), pred_latent[0,0,:,1,2].detach().numpy(), s=1)
axes[0][1].scatter(sample_batch['v'][0,0,:,2,1].detach().numpy(), pred_latent[0,0,:,2,2].detach().numpy(), s=1)
axes[0][1].scatter(sample_batch['v'][0,0,:,3,1].detach().numpy(), pred_latent[0,0,:,3,2].detach().numpy(), s=1)
axes[0][1].set_xlabel("v")
axes[0][1].set_ylabel("$v^{pred}$")
axes[0][1].set_title("$v_y$")
axes[1][0].scatter(sample_batch['v'][0,0,:,0,2].detach().numpy(), pred_latent[0,0,:,0,3].detach().numpy(), s=1)
axes[1][0].scatter(sample_batch['v'][0,0,:,1,2].detach().numpy(), pred_latent[0,0,:,1,3].detach().numpy(), s=1)
axes[1][0].scatter(sample_batch['v'][0,0,:,2,2].detach().numpy(), pred_latent[0,0,:,2,3].detach().numpy(), s=1)
axes[1][0].scatter(sample_batch['v'][0,0,:,3,2].detach().numpy(), pred_latent[0,0,:,3,3].detach().numpy(), s=1)
axes[1][0].set_xlabel("v")
axes[1][0].set_ylabel("$v^{pred}$")
axes[1][0].set_title("$v_x$")

plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(8, 7))
sc0 = axes[0][0].scatter(sample_batch['v'][0,0,:,0,0].detach().numpy(), pred_latent[0,0,:,0,1].detach().numpy(), s=1, c=sample_batch['x'][0,0,:,0])
plt.colorbar(sc0, ax=axes[0][0])
sc1 = axes[0][1].scatter(sample_batch['v'][0,0,:,1,0].detach().numpy(), pred_latent[0,0,:,1,1].detach().numpy(), s=1, c=sample_batch['x'][0,0,:,0])
plt.colorbar(sc1, ax=axes[0][1])
sc2 = axes[1][0].scatter(sample_batch['v'][0,0,:,2,0].detach().numpy(), pred_latent[0,0,:,2,1].detach().numpy(), s=1, c=sample_batch['x'][0,0,:,0])
plt.colorbar(sc2, ax=axes[1][0])
sc3 = axes[1][1].scatter(sample_batch['v'][0,0,:,3,0].detach().numpy(), pred_latent[0,0,:,3,1].detach().numpy(), s=1, c=sample_batch['x'][0,0,:,0])
plt.colorbar(sc3, ax=axes[1][1])

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1,1,figsize=(3.5, 3.5))
axes.scatter(sample_batch['h'][0,0,:,0].detach().numpy(), pred_latent[0,0,:,0,0].detach().numpy(), s=1)
axes.scatter(sample_batch['h'][0,0,:,1].detach().numpy(), pred_latent[0,0,:,1,0].detach().numpy(), s=1)
axes.scatter(sample_batch['h'][0,0,:,2].detach().numpy(), pred_latent[0,0,:,2,0].detach().numpy(), s=1)
axes.scatter(sample_batch['h'][0,0,:,3].detach().numpy(), pred_latent[0,0,:,3,0].detach().numpy(), s=1)
axes.set_xlabel("h")
axes.set_ylabel("$h^{pred}$")


plt.tight_layout()

In [ ]:
raise RuntimeError

## Decoder

In [ ]:

from argparse import Namespace
args = Namespace(
    sim_ckpt="workdir/rcut3.5_decoder_embeddim4_perturbeddata/epoch=619-step=207700.ckpt",
    # sim_ckpt="workdir/rcut3.5_energy_encodedim4/epoch=739-step=247900.ckpt",
    data_dir="test/rcut3.5_energy_encodedim4_perturbeddata/encoded_dataset-clean/",
    suffix="",
    num_frames=1,
    num_rollouts=1,
    # out_dir="./test/localmask_rcut3.5_loss_regress-path_linearOT",
    out_dir="./test/rcut3.5_decoder_embeddim4_perturbeddata",
    random_starting_point=True,
    localmask=False,
    )

device='cuda'
from mdgen.decoder_wrapper import DecoderWrapper
ckpt = torch.load(args.sim_ckpt, weights_only=False)
model = DecoderWrapper(**ckpt["hyper_parameters"])
print(model.model)
model.load_state_dict(ckpt["state_dict"], strict=False)
model.eval().to(device)


In [ ]:
pred_h = sample_batch["h"] # pred_latent[:,:,:,:,0]
pred_v = sample_batch['v']
pred_v = pred_latent[:,:,:,:,1:]

In [ ]:
out_dict = model.model(pred_h.to(device), pred_v.to(device))

In [ ]:
import numpy as np
def lattice_vectors(a, b, c, alpha, beta, gamma):
    # Convert angles to radians
    alpha_rad = np.radians(alpha)
    beta_rad = np.radians(beta)
    gamma_rad = np.radians(gamma)
    
    # Calculate vector components
    a_vec = np.array([a, 0, 0])
    b_vec = np.array([b * np.cos(gamma_rad), b * np.sin(gamma_rad), 0])
    
    cx = c * np.cos(beta_rad)
    cy = c * (np.cos(alpha_rad) - np.cos(beta_rad) * np.cos(gamma_rad)) / np.sin(gamma_rad)
    cz = np.sqrt(c**2 - cx**2 - cy**2)
    c_vec = np.array([cx, cy, cz])
    
    return a_vec, b_vec, c_vec

In [ ]:
cell = lattice_vectors(out_dict['cell'][0,0,0].item(), out_dict['cell'][0,0,1].item(), out_dict['cell'][0,0,2].item(), out_dict['cell'][0,0,3].item(), out_dict['cell'][0,0,4].item(), out_dict['cell'][0,0,5].item())

In [ ]:
print(out_dict['cell'][0,0,:])

In [ ]:

map_to_chemical_symbol = {
    0: "Cr",
    1: 'Co',
    2: "Ni"
}
aatype = out_dict['aatype'][0,0].argmax(dim=-1).cpu().numpy()
symbols = [map_to_chemical_symbol[i] for i in aatype]
print(symbols)

In [ ]:
frac_pos = out_dict['frac_pos'][0,0].detach().cpu().numpy()
pos = frac_pos @ np.array(cell)
print(pos.shape)

In [ ]:
print(frac_pos)
print(pos)

In [ ]:
formula = "".join(symbols)
from ase import Atoms
atoms = Atoms(formula, positions=pos, cell=np.eye(3), pbc=True)
from ase.io import write
write(f"{args.out_dir}/test.xyz", atoms, format="xyz")

## Training

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.model.parameters(), lr=0.001)
loss_epochs = []

In [ ]:
print(model.transport.model_type)

In [ ]:
# Training loop

num_epochs = 20
for epoch in range(num_epochs):
    Step = 0
    # for batch in train_loader:
    #     # Forward pass
    #     batch_ = {key: value.to(device) if isinstance(value, torch.Tensor) else value 
    #             for key, value in batch.items()}
    #     prep = model.prep_batch(batch_)
    # out_dict = model.transport.training_losses(
    #     model=model.model,
    #     x1=prep['latents'],
    #     aatype1=prep['species'],
    #     mask=prep['loss_mask'],
    #     model_kwargs=prep['model_kwargs']
    # )
    # loss = out_dict["loss"].mean()
    t = torch.ones(batch_size)
    out_dict = model.model(prep["latents"], t, **prep["model_kwargs"] )
    loss = ((energy - prep["E"])**2).mean()
    
    # Backward pass and optimize
    optimizer.zero_grad()
    # print("Starting backward")
    loss.backward()
    # print("Done")
    optimizer.step()
    Step += 1
    loss_epochs.append(loss.item())
    # print(f'Epoch [{epoch+1}/{num_epochs}], Step [{Step}], Loss: {loss.item():.4f}')
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(4,3.5))
plt.plot(loss_epochs)
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.semilogy()

## Testing the trained model

In [ ]:
t, x0, x1 = model.transport.sample(prep['latents'])
t, xt, ut = model.transport.path_sampler.plan(t, x0, x1)
out_dict = model.transport.training_losses(
    model=model.model,
    x1=prep['latents'],
    aatype1=prep['species'],
    mask=prep['loss_mask'],
    model_kwargs=prep['model_kwargs']
)
loss_last = out_dict["loss_continuous"][torch.where(prep['loss_mask'].squeeze(-1)!=0)].detach().mean(dim=1)
loss_fkl_last = out_dict["loss_fisherreg"]
print(loss_last)
import matplotlib.pyplot as plt
plt.figure(figsize=(4,3.5))
plt.scatter(t.detach().cpu().numpy(), loss_last.detach().cpu().numpy())
plt.scatter(t.detach().cpu().numpy(), loss_fkl_last.detach().cpu().numpy())
plt.xlabel("Diffusion time")
plt.ylabel("Loss")

In [ ]:
plt.figure(figsize=(4,3.5))
plt.scatter(t.detach().cpu().numpy(), loss_last.detach().cpu().numpy())
plt.xlabel("Diffusion time")
plt.ylabel("Loss")